In [1]:
import torch
import json
import os
import sys
import importlib
sys.path.append("..")
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.generation.logits_process import LogitsProcessorList, InfNanRemoveLogitsProcessor

from transformers_gad.grammar_utils import IncrementalGrammarConstraint
from transformers_gad.generation.logits_process import GrammarAlignedOracleLogitsProcessor

In [2]:
DATASET = "ebmoon/GAD-dataset"
# SPLIT = "SLIA"
SPLIT = "BV4"
# SPLIT = "CP"

NUM_ITER = 2000
# MODEL_ID = "mistralai/mistral-7b-instruct-v0.2"
MODEL_ID = "TinyLlama/TinyLlama_v1.1"
TRIE_PATH = f"tries/{SPLIT}"
RESULT_PATH = f"results/{SPLIT}"
DEVICE = "cuda"
# DEVICE = "cpu"
DTYPE = torch.bfloat16
MAX_NEW_TOKENS = 512
TEMPERATURE = 1.0
REPETITION_PENALTY = 1.0
TOP_P = 1.0
TOP_K = 0
MCMC_ITER_NUM = 10

In [3]:
device = torch.device(DEVICE)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(MODEL_ID)
model.to(device)
model.to(dtype=DTYPE)
model.resize_token_embeddings(len(tokenizer))

dataset = load_dataset(DATASET, split=SPLIT)

/home/jiry/.conda/envs/GAD/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/jiry/.conda/envs/GAD/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/jiry/.conda/envs/GAD/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.

In [4]:
task = dataset[0]
print(task['prompt'])

You are an expert in program synthesis. You are tasked with solving a Syntax-Guided Synthesis (SyGuS) problem. Your goal is to output a function that should produce outputs that satisfy a series of constraints when given specific inputs.

Question:
(set-logic BV)

(synth-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4)
    ((Start (BitVec 4)))
    ((Start (BitVec 4) (s t #x0 #x8 #x7 (bvneg Start) (bvnot Start) (bvadd Start Start) (bvsub Start Start) (bvand Start Start) (bvlshr Start Start) (bvor Start Start) (bvshl Start Start)))))

(declare-var s (BitVec 4))
(declare-var t (BitVec 4))
(define-fun udivtotal ((a (BitVec 4)) (b (BitVec 4))) (BitVec 4)
    (ite (= b #x0) #xF (bvudiv a b)))
(define-fun uremtotal ((a (BitVec 4)) (b (BitVec 4))) (BitVec 4)
    (ite (= b #x0) a (bvurem a b)))
(define-fun min () (BitVec 4)
    (bvnot (bvlshr (bvnot #x0) #x1)))
(define-fun max () (BitVec 4)
    (bvnot min))
(define-fun l ((s (BitVec 4)) (t (BitVec 4))) Bool
    (bvsle (bvnot (inv s t)) t))
(d

In [5]:
import transformers_gad
importlib.reload(transformers_gad)
from transformers_gad.generation.mcmc_process import BeamMCMC
grammar = IncrementalGrammarConstraint(task["grammar"], "root", tokenizer)

# Tokenize prompt into ids
input_ids = tokenizer(
    [task["prompt"]], add_special_tokens=False, return_tensors="pt", padding=True
)["input_ids"]
input_ids = input_ids.to(model.device)

# Inference Loop
outputs = []
history = []

def generator(processor):
    inf_nan_remove_processor = InfNanRemoveLogitsProcessor()
    logits_processors = LogitsProcessorList([
        inf_nan_remove_processor,
        processor,
    ])
    return model.generate(
        input_ids,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=MAX_NEW_TOKENS,
        top_p=TOP_P,
        top_k=TOP_K,
        temperature=TEMPERATURE,
        logits_processor=logits_processors,
        repetition_penalty=REPETITION_PENALTY,
        num_return_sequences=1,
        # return_dict_in_generate=True,
        # output_scores=True,
    )[0]

def beam_search(processor, beam_num):
    inf_nan_remove_processor = InfNanRemoveLogitsProcessor()
    logits_processors = LogitsProcessorList([
        inf_nan_remove_processor,
        processor,
    ])
    return model.generate(
        input_ids,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=MAX_NEW_TOKENS,
        temperature=TEMPERATURE,
        logits_processor=logits_processors,
        repetition_penalty=REPETITION_PENALTY,
        num_beams=beam_num,
        num_return_sequences=beam_num,  # Number of sequences to return
        early_stopping=True
        # return_dict_in_generate=True,
        # output_scores=True,
    )

holder = BeamMCMC(input_ids[0], generator, beam_search, tokenizer, grammar, DEVICE, beam_num=5, beam_weight=0.8)

In [6]:
import tqdm
for _ in tqdm.tqdm(range(10)):
    print(holder.mcmc(100))

  0%|                                                                                                                                                                                                        | 0/10 [00:00<?, ?it/s]

current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) #x0)</s> 6.171046594656827e-10
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) #x8)</s> 5.2074884213676594e-11
  acc 0.09864672927162366
accepted
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) #x8)</s> 5.2074884213676594e-11
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvlshr t s))</s> 8.790139525835277e-21
  acc 1.321293195790231e-12
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) #x8)</s> 5.2074884213676594e-11
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg #x0))</s> 4.093259800052119e-15
  acc 2.7077246593734787e-07
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) #x8)</s> 5.2074884213676594e-11
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvlshr t #x0))</s> 1.803629443805292e-17
  acc 1.2975696739529339e-09
current node (define-fun inv ((s (BitVec 4)) (t (BitVec

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvlshr t #x0))</s> 2.6547285439878124e-08
  acc 1.0
accepted
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvlshr t #x0))</s> 2.6547285439878124e-08
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  acc 1.0
accepted
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvlshr t #x0))</s> 2.6547285439878124e-08
  acc 0.013326620347514978
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg #x0))</s> 4.213186615106936e-17
  acc 1.3443100623396071e-11
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitVe

 10%|███████████████████                                                                                                                                                                            | 1/10 [04:59<44:53, 299.31s/it]

...

  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg #x0))</s> 2.317790628040093e-28
  acc 7.100852465941022e-21
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) #x0)</s> 3.000939383799428e-14
  acc 3.4306932976192142e-09
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 1.2402186655650561e-12
  acc 5.449302997563746e-07
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvshl t #x0))</s> 2.3050845312843784e-09
  acc 0.0030046112694119133
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitV

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                   | 9/10 [06:37<00:13, 13.30s/it]

  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) #x0)</s> 6.965120001909916e-25
  acc 3.7841609728689734e-19
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg #x0))</s> 4.093259800052119e-15
  acc 5.295212538593237e-11
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvlshr t s))</s> 1.2122148231366822e-20
  acc 3.098433778666831e-16
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg #x0))</s> 7.797402493749859e-21
  acc 2.582495582831418e-16
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitV

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:45<00:00, 40.52s/it]

  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) #x0)</s> 3.928319796925229e-27
  acc 7.510145513511272e-21
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg #x0))</s> 4.093259800052119e-15
  acc 5.295212538593237e-11
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 1.2402186655650561e-12
  acc 5.449302997563746e-07
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) #x0)</s> 6.171046594656827e-10
  acc 4.395019948443607e-05
current node (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (BitVec 4) (bvneg t))</s> 2.275921647446524e-06
  to (define-fun inv ((s (BitVec 4)) (t (BitVec 4))) (Bit